## The imports

In [1]:
import zarr
import xarray as xr
import time
import dask
from intake import open_catalog


# The workers

In [2]:
from dask.distributed import Client
from dask_gateway import Gateway

gateway = Gateway()
cluster = gateway.new_cluster()
cluster.scale(40)
cluster


In [3]:
c = Client(cluster)
c

Client Scheduler: gateway://traefik-gcp-uscentral1b-prod-dask-gateway.prod:80/prod.ab975260b03842938afa18332614c4ea Dashboard: /services/dask-gateway/clusters/prod.ab975260b03842938afa18332614c4ea/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [4]:
from dask.utils import ensure_dict, format_bytes
    
wk = c.scheduler_info()["workers"]

text="Workers= " + str(len(wk))
memory = [w["memory_limit"] for w in wk.values()]
cores = sum(w["nthreads"] for w in wk.values())
text += ", Cores=" + str(cores)
if all(memory):
    text += ", Memory=" + format_bytes(sum(memory))
print(text)


Workers= 40, Cores=80, Memory=343.60 GB


# The data

In [5]:
%%time 

import xarray as xr
import gcsfs

fs = gcsfs.GCSFileSystem('pangeo-181919', requester_pays=True)

zmap = fs.get_mapper('pangeo-meom/autre-eNATL60-BLBT02-SSH-1h/eNATL60-BLBT02-SSH-1h')
ds = xr.open_zarr(zmap)


CPU times: user 1.12 s, sys: 205 ms, total: 1.33 s
Wall time: 6.95 s


In [6]:
%time mean=ds.sossheig.mean(dim='time_counter')

CPU times: user 81.9 ms, sys: 6.87 ms, total: 88.8 ms
Wall time: 86.1 ms


In [ ]:
%time mean.load()

In [ ]:
cluster.close()